# Análise das denúncias de poluição sonora
Exploração tabular do arquivo `mga_denuncias_20-23.geojson`, com foco nos campos gerados pelo pipeline de NLP (tokens, contexto, modalidade e horários).

In [1]:
import json
from pathlib import Path

import pandas as pd

GEOJSON_PATH = Path("mga_denuncias_20-23.geojson")

with GEOJSON_PATH.open() as fh:
    geojson = json.load(fh)

df = pd.json_normalize(
    geojson["features"],
    record_path=None,
    meta=["type"],
)
properties_cols = [col for col in df.columns if col.startswith("properties.")]
df_properties = df[properties_cols].rename(columns=lambda c: c.removeprefix("properties."))
df_properties.head()

,Protocolo,Endereço,Número,Bairro,Zona,Setor,DataInclusao,Cadastro,Assunto,Descrição,...,tipo_fonte_score,tipo_fonte_termos,fonte_cluster,fonte_contexto,fonte_contexto_score,fonte_contexto_termos,fonte_audio,fonte_audio_score,fonte_audio_termos,fonte_horario
0,649554,Alameda Amor Perfeito,36,Zona 04 - centro,60,IAM - Fiscalização,2022-11-14 22:20:00,60130700,Poluição sonora por estabelecimento comercial,Som ao vivo fora do estabelecimento provocando...,...,2,"[som, vivo]",cluster_2,bar_evento,1,[estabelecimento],musica,2,"[som, vivo]",[manha]
1,601042,Alameda Das Alfazemas,1968,Zona 3,60,IAM - Fiscalização,2022-05-21 23:54:00,60067500,Poluição sonora por estabelecimento comercial,Muito barulho som alto de dois bares thel bar ...,...,1,[som],cluster_0,bar_evento,1,[bar],musica,1,[som],[]
2,660699,Alameda Doutor João Paulino,673,JARDIM ALVORADA I PARTE,24,IAM - Fiscalização,2022-12-23 23:49:00,24130900.0,Poluição sonora por estabelecimento comercial,Vizinho vários dias com som alto além da hora....,...,1,[som],cluster_0,residencial,2,"[casa, vizinho]",musica,1,[som],[noite]
3,784845,Avenida Advogado Horácio Raccanello Filho,183,Vila nova,18,IAM - Fiscalização,2023-12-23 08:56:00,18039700.0,Poluição sonora por estabelecimento comercial,Esse estabelecimento permanece com o som alto ...,...,2,"[música, som]",cluster_2,bar_evento,2,"[estabelecimento, residencial]",musica,2,"[música, som]",[]
4,648166,Avenida Advogado Horácio Raccanello Filho,3605,ZONA ARMAZÉM,9,IAM - Fiscalização,2022-11-09 08:45:00,9014300.0,Poluição sonora por estabelecimento comercial,na madrugada de terça para quarta (dia 08 para...,...,1,[som],cluster_2,,0,[],musica,1,[som],"[madrugada, fim_de_semana]"


In [2]:
df_properties[[
    "Protocolo",
    "Bairro",
    "Descrição",
    "descricao_tokens",
    "fonte_contexto",
    "fonte_audio",
    "Tipo de Fonte",
    "fonte_horario",
]].head(10)

,Protocolo,Bairro,Descrição,descricao_tokens,fonte_contexto,fonte_audio,Tipo de Fonte,fonte_horario
0,649554,Zona 04 - centro,Som ao vivo fora do estabelecimento provocando...,"[som, vivo, estabelecimento, provocar, incômod...",bar_evento,musica,bar_evento_musica,[manha]
1,601042,Zona 3,Muito barulho som alto de dois bares thel bar ...,"[som, alto, bar, thel, bar, espetinho, insupor...",bar_evento,musica,bar_evento_musica,[]
2,660699,JARDIM ALVORADA I PARTE,Vizinho vários dias com som alto além da hora....,"[vizinho, vários, dia, som, alto, além, hora, ...",residencial,musica,residencial_musica,[noite]
3,784845,Vila nova,Esse estabelecimento permanece com o som alto ...,"[estabelecimento, permanecer, som, alto, após,...",bar_evento,musica,bar_evento_musica,[]
4,648166,ZONA ARMAZÉM,na madrugada de terça para quarta (dia 08 para...,"[madrugada, terça, quarta, dia, novembro, tend...",,musica,musica,"[madrugada, fim_de_semana]"
5,649328,ZONA ARMAZÉM,batucada religiosa fora de contesto.,"[batucada, religioso, contesto]",,,indefinido,[]
6,656895,ZONA ARMAZÉM,Agora sao 23:20h do dia 10/12 e a tenda de umb...,"[agora, sao, dia, tenda, umbanda, amigo, axé, ...",,musica,musica,[fim_de_semana]
7,674293,ZONA ARMAZÉM,agora são 22;48 do dia 06/02/2023 (segunda fei...,"[agora, dia, segundo, feira, tenda, umbanda, c...",,musica,musica,[fim_de_semana]
8,704468,ZONA ARMAZÉM,a tenda de umbanda novamente esta desrepeitand...,"[tenda, umbanda, novamente, desrepeitar, lei, ...",,musica,musica,[fim_de_semana]
9,648219,ZONA 3,Por gentileza notificar essa Tenda de Umbanda ...,"[gentileza, notificar, tenda, umbanda, amigo, ...",bar_evento,musica,bar_evento_musica,[madrugada]


In [3]:
summary = (
    df_properties["Tipo de Fonte"]
    .value_counts()
    .rename_axis("Tipo de Fonte")
    .reset_index(name="Denúncias")
)
summary

,Tipo de Fonte,Denúncias
0,bar_evento_musica,1245
1,musica,344
2,bar_evento,155
3,residencial_musica,148
4,indefinido,90
5,via_publica_musica,80
6,obra_construcao,55
7,bar_evento_vozes_aglomeracao,32
8,igreja_templo_musica,30
9,area_lazer_musica,25


In [4]:
cross_context_audio = (
    df_properties.groupby(["fonte_contexto", "fonte_audio"]).size().reset_index(name="Denúncias")
)
cross_context_audio.sort_values("Denúncias", ascending=False).head(20)

,fonte_contexto,fonte_audio,Denúncias
15,bar_evento,musica,1245
5,,musica,344
10,bar_evento,,155
39,residencial,musica,148
0,,,90
51,via_publica,musica,80
29,obra_construcao,,55
17,bar_evento,vozes_aglomeracao,32
19,igreja_templo,musica,30
9,area_lazer,musica,25


In [5]:
horario_explode = df_properties.explode("fonte_horario")
horario_summary = (
    horario_explode["fonte_horario"].dropna().value_counts().rename_axis("Horário").reset_index(name="Denúncias")
)
horario_summary

,Horário,Denúncias
0,noite,559
1,fim_de_semana,549
2,manha,465
3,madrugada,425
4,tarde,170
